In [1]:
import numpy as np
from pathlib import Path


In [2]:
with Path("../14.in").open() as f:
    data = f.read().splitlines()
data = np.array([list(row) for row in data])


In [3]:
# "O" is a movable rock
# "#" is a stationary rock
# "." is free space

testdata = """\
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....""".splitlines()
testdata = np.array([list(row) for row in testdata])


## Part I

In [4]:
direction_to_rotations = {
    "north": 0,
    "east": 1,
    "south": 2,
    "west": 3
}


def tilt_dish(dish, direction="north"):
    rotations = direction_to_rotations[direction]
    rotated_dish = np.rot90(dish, k=rotations)

    for ci in range(rotated_dish.shape[1]):
        last_free = -1
        for ri in range(rotated_dish.shape[0]):
            curr = rotated_dish[ri, ci]
            if curr == ".":
                if last_free == -1:
                    last_free = ri
            elif curr == "#":
                last_free = -1
            elif curr == "O":
                if last_free != -1:
                    rotated_dish[last_free, ci] = "O"
                    rotated_dish[ri, ci] = "."
                    last_free += 1

    modified_dish = np.rot90(rotated_dish, k=-rotations)
    return modified_dish


def calculate_load(dish, direction="north"):
    rotations = direction_to_rotations[direction]
    rotated_dish = np.rot90(dish, k=rotations)
    total_load = 0

    rocks_per_row = np.sum(rotated_dish == "O", axis=1)
    total_load = np.sum(rocks_per_row * np.arange(1, rotated_dish.shape[1]+1)[::-1])

    return total_load


def solve1(dish):
    modified_dish = tilt_dish(dish.copy(), direction="north")
    total_load = calculate_load(modified_dish, direction="north")
    print(f"Part 1: {total_load}")
    return total_load

assert solve1(testdata) == 136


Part 1: 136


In [5]:
solve1(data)


Part 1: 109755


109755

## Part II

In [6]:

def solve2(dish, cycles=1_000_000_000, verbose=False):
    current_dish = dish.copy()
    directions = ["north", "west", "south", "east"]

    if verbose:
        dish_hash = hash(current_dish.data.tobytes())
        load = calculate_load(current_dish, direction="north")
        print("Initial dish:")
        print(f"\t{dish_hash=}\n\t{load=}")
        print(current_dish)
        print()

    cache_data = {}
    cache_idx = {}
    for i in range(cycles):

        # cache input to the function
        key = current_dish.data.tobytes()

        if key in cache_data:
            cycle_start = cache_idx[key]
            cycle_end = i
            cycle_length = cycle_end - cycle_start
            print(f"Cycle found at {cycle_start}->{cycle_end} with length {cycle_length}!")
            break
            # continue

        for direction in directions:
            current_dish = tilt_dish(current_dish, direction=direction)

        if verbose:
            dish_hash = hash(current_dish.data.tobytes())
            load = calculate_load(current_dish, direction="north")
            print(f"After cycle {i}:")
            print(f"\t{dish_hash=}\n\t{load=}")
            print(current_dish)
            print()

        # cache output of the function
        cache_data[key] = current_dish
        cache_idx[key] = i

    print("------------------------------------------")
    leftover_cycles = cycles - cycle_end
    print(f"Leftover cycles: {leftover_cycles}")

    # Since the cycle is known
    for i in range(leftover_cycles % cycle_length):
        for direction in directions:
            current_dish = tilt_dish(current_dish, direction=direction)

    if verbose:
        dish_hash = hash(current_dish.data.tobytes())
        load = calculate_load(current_dish, direction="north")
        print("After final cycle:")
        print(f"\t{dish_hash=}\n\t{load=}")
        print(current_dish)
        print()


    total_load = calculate_load(current_dish, direction="north")
    print(f"Part 2: {total_load}")
    return total_load


assert solve2(testdata, cycles=1_000_000_000, verbose=True) == 64


Initial dish:
	dish_hash=2753347076773054734
	load=104
[['O' '.' '.' '.' '.' '#' '.' '.' '.' '.']
 ['O' '.' 'O' 'O' '#' '.' '.' '.' '.' '#']
 ['.' '.' '.' '.' '.' '#' '#' '.' '.' '.']
 ['O' 'O' '.' '#' 'O' '.' '.' '.' '.' 'O']
 ['.' 'O' '.' '.' '.' '.' '.' 'O' '#' '.']
 ['O' '.' '#' '.' '.' 'O' '.' '#' '.' '#']
 ['.' '.' 'O' '.' '.' '#' 'O' '.' '.' 'O']
 ['.' '.' '.' '.' '.' '.' '.' 'O' '.' '.']
 ['#' '.' '.' '.' '.' '#' '#' '#' '.' '.']
 ['#' 'O' 'O' '.' '.' '#' '.' '.' '.' '.']]

After cycle 0:
	dish_hash=-9128443251630661766
	load=87
[['.' '.' '.' '.' '.' '#' '.' '.' '.' '.']
 ['.' '.' '.' '.' '#' '.' '.' '.' 'O' '#']
 ['.' '.' '.' 'O' 'O' '#' '#' '.' '.' '.']
 ['.' 'O' 'O' '#' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' 'O' 'O' 'O' '#' '.']
 ['.' 'O' '#' '.' '.' '.' 'O' '#' '.' '#']
 ['.' '.' '.' '.' 'O' '#' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' 'O' 'O' 'O' 'O']
 ['#' '.' '.' '.' 'O' '#' '#' '#' '.' '.']
 ['#' '.' '.' 'O' 'O' '#' '.' '.' '.' '.']]

After cycle 1:
	dish_hash

In [7]:
solve2(data, cycles=1_000_000_000, verbose=False)

Cycle found at 153->179 with length 26!
------------------------------------------
Leftover cycles: 999999821
Part 2: 90928


90928